In [12]:
import os
import pandas as pd
import glob
import json
version = "v4"
data_path = f"../data/interim/{version}"
dataset_target_path = "../data/processed"

In [2]:
from lib import read_dataset
data_on, data_off = read_dataset(data_path, classes = 2)

0 milenkosKitchenNotNearDevicesWithTraffic
1 milenkosKitchenNotNearDevicesWithTraffic1
2 milenkoParkWithTraffic1 
3 milenkosRoomNearDevicesWithTraffic
4 milenkosRoomNearDevicesWithTraffic1
5 milenkoParkWithTraffic


In [3]:
from lib import add_intensity_to_dataset
add_intensity_to_dataset(data_on)
add_intensity_to_dataset(data_off)

In [4]:
from lib import statistical_dataset
# all_on is class 6
dataset_on = statistical_dataset(data_on, label=6)
# all_of is class 5
dataset_off = statistical_dataset(data_off, label=5)

In [5]:
from lib import join_and_order_dataset
data_statistical = join_and_order_dataset([dataset_on, dataset_off])
data_statistical

,Name,Label,count,X_UnCal_mean,X_UnCal_std,X_UnCal_min,X_UnCal_25%,X_UnCal_50%,X_UnCal_75%,X_UnCal_max,...,Intensity_min,Intensity_25%,Intensity_50%,Intensity_75%,Intensity_max,X_Bias,Y_Bias,Z_Bias,Bias_Intensity,Accuracy
0,milenkoParkWithTraffic,5,5998.0,-36.101823,1.097864,-47.872800,-36.795200,-36.1852,-35.40440,-32.781400,...,100.995317,102.845848,103.315567,103.746563,109.228455,-17.414808,48.837105,-32.695650,61.297177,1
1,milenkoParkWithTraffic1,5,5998.0,-36.101823,1.097864,-47.872800,-36.795200,-36.1852,-35.40440,-32.781400,...,100.995317,102.845848,103.315567,103.746563,109.228455,-17.414808,48.837105,-32.695650,61.297177,1
2,milenkosKitchenNotNearDevicesWithTraffic,5,5998.0,1.603930,1.136567,-8.625400,0.902800,1.5250,2.26920,5.038600,...,80.865373,82.649076,83.028101,83.407196,88.664891,-15.227382,46.575470,-33.300674,59.245949,1
3,milenkosKitchenNotNearDevicesWithTraffic1,5,5998.0,1.714448,1.180194,-9.308599,0.988200,1.6104,2.42780,5.087400,...,81.086756,82.926652,83.312701,83.714210,89.914525,-15.227382,46.575470,-33.300674,59.245949,1
4,milenkosRoomNearDevicesWithTraffic,5,5998.0,18.273901,1.208670,9.076799,17.531399,18.2390,19.06860,21.752600,...,91.842454,93.567313,93.908219,94.276252,98.699186,-19.871798,47.590263,-31.834210,60.606422,1
5,milenkosRoomNearDevicesWithTraffic1,5,5998.0,18.126674,1.265258,10.638400,17.388050,18.0926,18.91915,21.813599,...,91.590948,93.494043,93.860412,94.238884,99.375866,-19.871798,47.590263,-31.834210,60.606422,1
6,milenkoParkWithTraffic,6,6002.0,-33.455725,1.638925,-54.424200,-33.794000,-33.2206,-32.69600,-25.973799,...,94.706865,101.452069,101.899838,102.344849,117.805710,-17.414808,48.837105,-32.695650,61.297177,1
7,milenkoParkWithTraffic1,6,6002.0,-33.455725,1.638925,-54.424200,-33.794000,-33.2206,-32.69600,-25.973799,...,94.706865,101.452069,101.899838,102.344849,117.805710,-17.414808,48.837105,-32.695650,61.297177,1
8,milenkosKitchenNotNearDevicesWithTraffic,6,5998.0,4.335766,2.052637,-18.226799,4.160200,4.8556,5.36800,8.113000,...,79.047434,81.736886,82.196564,82.677931,94.061595,-15.227382,46.575470,-33.300674,59.245949,1
9,milenkosKitchenNotNearDevicesWithTraffic1,6,6001.0,4.770478,1.834772,-10.931200,4.379800,5.2948,5.83160,9.516000,...,72.568444,82.388501,82.779071,83.200768,95.330136,-15.227382,46.575470,-33.300674,59.245949,1


In [18]:
import time
from lib import save_dataset
# metadata
with open(f"{data_path}/metadata.json", 'r') as jsonfile:
    metadata = json.load(jsonfile)
    metadata['title'] = f"{version} statistical dataset"
    metadata['description'] = 'Dataset with statistical features from '+metadata['description']
    metadata['format'] = "processed"
    metadata['author'] = 'Mihael'
    metadata['version'] = version
    metadata['created'] = int(time.time())
    save_dataset(data_statistical, dataset_target_path, dataset_name = 'statistical', metadata=metadata)

